In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import re
import nltk
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
%matplotlib inline

# read csv file
df = pd.read_csv("H-1B_Disclosure_Data_FY2019.csv")

## caculate time delta need to find package
time_span = df.loc[:, ["CASE_SUBMITTED", 'DECISION_DATE']]
time_span.fillna(0, inplace = True)

# REMOVE ROWS
df_new = df[df['CASE_STATUS'].notnull()]
df_new = df_new[(df_new['CASE_STATUS'] == 'CERTIFIED') | (df_new['CASE_STATUS'] == 'DENIED')]
df_new = df_new[df_new['VISA_CLASS'] == 'H-1B']
df_new = df_new[df_new['EMPLOYER_STATE'].notnull()]

# drop columns with over 75% missing values
def clean(X):
    '''
    X: dataframe to be cleaned
    '''
    columns = X.columns
    drop = []
    for i in range(len(columns)):
        if X[columns[i]].isnull().sum() == len(X) or X[columns[i]].isnull().sum() / len(X[columns[i]]) > 0.75:
            drop.append(columns[i])
    return X.drop(columns = drop, axis = 1)

# remove columns with all null values
cleaned = clean(df_new)

# Undersampling
shuffled_cleaned = cleaned.sample(frac=1,random_state=4)
DENIED = shuffled_cleaned.loc[shuffled_cleaned['CASE_STATUS']=='DENIED']
CERTIFIED = shuffled_cleaned.loc[shuffled_cleaned['CASE_STATUS']=='CERTIFIED'].sample(n=len(DENIED)*3,random_state=42)
cleaned = pd.concat([DENIED, CERTIFIED])
cleaned = cleaned.reset_index(drop=True)

# before selecting columns let's make a new column (total number of empty columns)
for i in range(len(cleaned.index)):
    cleaned.loc[i,'NumOfNaN'] = cleaned.iloc[i].isnull().sum()
    
# Further Column cleaning: columns to retain include
includeColumns = ['CASE_STATUS','CASE_NUMBER','CASE_SUBMITTED','DECISION_DATE','SOC_CODE','FULL_TIME_POSITION'
                 ,'PERIOD_OF_EMPLOYMENT_START_DATE','PERIOD_OF_EMPLOYMENT_END_DATE','TOTAL_WORKER_POSITIONS','EMPLOYER_NAME'
                 ,'EMPLOYER_STATE','AGENT_REPRESENTING_EMPLOYER','WAGE_RATE_OF_PAY_FROM_1','WAGE_UNIT_OF_PAY_1','WAGE_RATE_OF_PAY_TO_1'
                 ,'H-1B_DEPENDENT','WILLFUL_VIOLATOR','NumOfNaN']
cleaned = cleaned[includeColumns]

cleaned.head()



C:\Users\tdann\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,13,14,19,20,21,22,23,24,25,26,27,28,29,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,48,49,50,51,52,53,54,55,57,58,59,61,64,66,67,68,69,70,71,72,73,76,78,79,80,82,85,87,88,89,90,91,92,93,94,97,98,99,100,101,103,106,108,109,110,111,112,113,114,115,121,122,124,127,129,130,131,132,133,134,135,136,142,143,145,148,150,151,152,153,154,155,156,157,160,162,163,164,166,169,171,172,173,174,175,176,177,178,184,185,187,190,192,193,194,195,196,197,198,199,205,206,208,211,213,214,215,216,217,218,219,220,223,225,226,227,229,232,234,235,236,237,238,239,240,244,246,247,248,250,253,254,255,256,257,258,259) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CASE_STATUS,CASE_NUMBER,CASE_SUBMITTED,DECISION_DATE,SOC_CODE,FULL_TIME_POSITION,PERIOD_OF_EMPLOYMENT_START_DATE,PERIOD_OF_EMPLOYMENT_END_DATE,TOTAL_WORKER_POSITIONS,EMPLOYER_NAME,EMPLOYER_STATE,AGENT_REPRESENTING_EMPLOYER,WAGE_RATE_OF_PAY_FROM_1,WAGE_UNIT_OF_PAY_1,WAGE_RATE_OF_PAY_TO_1,H-1B_DEPENDENT,WILLFUL_VIOLATOR,NumOfNaN
0,DENIED,I-200-19130-977369,6/5/2019 16:01,6/7/2019 12:15,11-9111,Y,9/11/2019,9/10/2022,1.0,"NEW CENTURY HOME HEALTH CARE, INC.",MI,Y,60341,Year,NaN,N,N,5.0
1,DENIED,I-200-19030-012395,1/30/2019 4:06,2/4/2019 10:32,13-1111,Y,2/22/2019,12/3/2021,1.0,CAPGEMINI AMERICA INC,IL,N,108900,Year,NaN,Y,N,12.0
2,DENIED,I-200-18309-480955,11/08/2018 15:19:26,11/13/2018 09:08:24,15-1132,Y,04/27/2019,04/26/2022,1.0,"MACY'S SYSTEMS AND TECHNOLOGY, INC.",CA,Y,114400,Year,NaN,N,N,10.0
3,DENIED,I-200-18288-204824,10/15/2018 10:27:56,10/17/2018 10:23:23,11-9121,N,04/04/2019,04/03/2022,1.0,"FERMI RESEARCH ALLIANCE, LLC",IL,N,63.83,Hour,94.57,N,N,18.0
4,DENIED,I-200-19073-344443,3/14/2019 15:02,3/19/2019 11:59,51-8091,Y,4/15/2019,4/15/2022,1.0,AGRAFORM,MO,N,20,Hour,28.5,N,N,17.0


In [2]:
# Further Cleaning (Modifying columns into meaningful columns)

# 1. Merge SOC_CODE column with SOC_CODE file from US government to make new column unified SOC_TITLE (observation<=23)
cleaned['SOC_CODE'] = cleaned['SOC_CODE'].str.replace(r"OPERATIONS RESEARCH ANALYSTS",'15')
cleaned['SOC_CODE'] = cleaned['SOC_CODE'].fillna('15')
cleaned['SOC_CODE'] = cleaned['SOC_CODE'].str.replace(r"-.*",'')
cleaned['SOC_CODE'] = pd.to_numeric(cleaned['SOC_CODE'], downcast="float")

# 2. Modify Case Submitted Date, Decision Date, PERIOD_OF_EMPLOYMENT_END_DATE and PERIOD_OF_EMPLOYMENT_START_DATE
import calendar
month_dict = dict(("-{}-".format(v),"/{}/".format(k)) for k,v in enumerate(calendar.month_abbr))
# same format time day/month/year
def dateFormat(X):
    for i, j in month_dict.items():
        if i in X:
            X = X.replace(i, j)
    X = X.split()[0]
    X = X.split('/')
    mon = X[0]
    day = X[1]
    year = X[2]
    if len(day) == 1:
        day = "0" + day
    if len(year) == 4:
        year = year[2:]
    return "".join(mon + "/" + day + "/" + year)

cleaned["CASE_SUBMITTED"] = cleaned["CASE_SUBMITTED"].apply(dateFormat)
cleaned["DECISION_DATE"] = cleaned["DECISION_DATE"].apply(dateFormat)
cleaned['PERIOD_OF_EMPLOYMENT_END_DATE'] = cleaned['PERIOD_OF_EMPLOYMENT_END_DATE'].apply(dateFormat)
cleaned['PERIOD_OF_EMPLOYMENT_START_DATE'] = cleaned['PERIOD_OF_EMPLOYMENT_START_DATE'].apply(dateFormat)

In [3]:
# 3. Calculate 'PERIOD_OF_EMPLOYMENT_LENGTH' from 'PERIOD_OF_EMPLOYMENT_END_DATE' and 'PERIOD_OF_EMPLOYMENT_START_DATE'
a = cleaned['PERIOD_OF_EMPLOYMENT_END_DATE'].str.findall(r'\d+$')
b = cleaned['PERIOD_OF_EMPLOYMENT_START_DATE'].str.findall(r'\d+$')
aa = np.array([int(i[0]) for i in a])
bb = np.array([int(i[0]) for i in b])
difference = list(abs(aa-bb))
cleaned['PERIOD_OF_EMPLOYMENT_LENGTH'] = difference
# drop old columns
cleaned.drop(columns = ['PERIOD_OF_EMPLOYMENT_END_DATE','PERIOD_OF_EMPLOYMENT_START_DATE'],inplace = True)

# 4. Create a wage column from WAGE_RATE_OF_PAY_FROM_1, WAGE_UNIT_OF_PAY_1 and WAGE_RATE_OF_PAY_TO_1

# clean up the dataset
cleaned['WAGE_RATE_OF_PAY_FROM_1'] = cleaned['WAGE_RATE_OF_PAY_FROM_1'].fillna(0)
cleaned['WAGE_RATE_OF_PAY_TO_1'] = cleaned['WAGE_RATE_OF_PAY_TO_1'].fillna(0)
cleaned['WAGE_RATE_OF_PAY_FROM_1'] = cleaned['WAGE_RATE_OF_PAY_FROM_1'].astype(str)
cleaned['WAGE_RATE_OF_PAY_TO_1'] = cleaned['WAGE_RATE_OF_PAY_TO_1'].astype(str)
cleaned['WAGE_RATE_OF_PAY_FROM_1']=cleaned['WAGE_RATE_OF_PAY_FROM_1'].str.replace(r"\$|\..*|,","")
cleaned['WAGE_RATE_OF_PAY_TO_1']=cleaned['WAGE_RATE_OF_PAY_TO_1'].str.replace(r"\$|\..*|,","")

def yearly_salary(row):

#     if row['WAGE_UNIT_OF_PAY_1'] == 'Year':
#         return max(float(row['WAGE_RATE_OF_PAY_FROM_1']),float(row['WAGE_RATE_OF_PAY_TO_1']))
    if (row['WAGE_UNIT_OF_PAY_1'] == 'Hour') & (len(row['WAGE_RATE_OF_PAY_FROM_1'])<=3) & (len(row['WAGE_RATE_OF_PAY_TO_1'])<=3):
        return max(float(row['WAGE_RATE_OF_PAY_FROM_1']),float(row['WAGE_RATE_OF_PAY_TO_1']))*2080
    elif (row['WAGE_UNIT_OF_PAY_1'] == 'Month') & (len(row['WAGE_RATE_OF_PAY_FROM_1'])<=5) & (len(row['WAGE_RATE_OF_PAY_TO_1'])<=5):
        return max(float(row['WAGE_RATE_OF_PAY_FROM_1']),float(row['WAGE_RATE_OF_PAY_TO_1']))*12
    elif (row['WAGE_UNIT_OF_PAY_1'] == 'Bi-Weekly') & (len(row['WAGE_RATE_OF_PAY_FROM_1'])<=5) & (len(row['WAGE_RATE_OF_PAY_TO_1'])<=5):
        return max(float(row['WAGE_RATE_OF_PAY_FROM_1']),float(row['WAGE_RATE_OF_PAY_TO_1']))*26
    elif (row['WAGE_UNIT_OF_PAY_1'] == 'Week') & (len(row['WAGE_RATE_OF_PAY_FROM_1'])<=4) & (len(row['WAGE_RATE_OF_PAY_TO_1'])<=4):
        return max(float(row['WAGE_RATE_OF_PAY_FROM_1']),float(row['WAGE_RATE_OF_PAY_TO_1']))*52
    else:
        return max(float(row['WAGE_RATE_OF_PAY_FROM_1']),float(row['WAGE_RATE_OF_PAY_TO_1']))
        
cleaned["Annual_Wage"] = cleaned.apply(yearly_salary,axis=1)
# drop old columns
deletecolumns = ['WAGE_RATE_OF_PAY_FROM_1','WAGE_RATE_OF_PAY_TO_1','WAGE_UNIT_OF_PAY_1']
cleaned.drop(columns = deletecolumns,inplace = True)



In [4]:
# 5. Column with month of petition submission during the fiscal year from Sep 2018 to Aug 2019
cleaned['CASE_SUBMITTED']=cleaned['CASE_SUBMITTED'].str.replace(r"/.*","")

# drop old column
cleaned.drop(columns = 'DECISION_DATE',inplace = True)
cleaned.head()

,CASE_STATUS,CASE_NUMBER,CASE_SUBMITTED,SOC_CODE,FULL_TIME_POSITION,TOTAL_WORKER_POSITIONS,EMPLOYER_NAME,EMPLOYER_STATE,AGENT_REPRESENTING_EMPLOYER,H-1B_DEPENDENT,WILLFUL_VIOLATOR,NumOfNaN,PERIOD_OF_EMPLOYMENT_LENGTH,Annual_Wage
0,DENIED,I-200-19130-977369,6,11.0,Y,1.0,"NEW CENTURY HOME HEALTH CARE, INC.",MI,Y,N,N,5.0,3,60341.0
1,DENIED,I-200-19030-012395,1,13.0,Y,1.0,CAPGEMINI AMERICA INC,IL,N,Y,N,12.0,2,108900.0
2,DENIED,I-200-18309-480955,11,15.0,Y,1.0,"MACY'S SYSTEMS AND TECHNOLOGY, INC.",CA,Y,N,N,10.0,3,114400.0
3,DENIED,I-200-18288-204824,10,11.0,N,1.0,"FERMI RESEARCH ALLIANCE, LLC",IL,N,N,N,18.0,3,195520.0
4,DENIED,I-200-19073-344443,3,51.0,Y,1.0,AGRAFORM,MO,N,N,N,17.0,3,58240.0


In [5]:
# 6. Make a new column using the Fortune 500: binary column with yes or no: yes means employer company is in the fortune 500
fortune = pd.read_csv("f500.csv",dtype= object)
compname = list(fortune.COMPANY)

def cleancompanyname(text):
    if text != 'DIVISION OF, LLC':
        text = str(text)
        y = text.lower()
        y = re.sub(r"\s+the\s+|\s+a\s+","",y)
        y = re.sub(r".*division of\s*|.*subsid*i*a*r*y* of\s*","",y)
        y = re.sub(r".*wos of\s*|.*trustees of\s*","",y)
        y = re.sub(r".*part of\s*|.*services of\s*","",y)
        y = re.sub(r"\s*inc\.*\s*|\s*llc\.*\s*","",y)
        y = re.sub(r"\s+corp\.*o*r*a*t*i*o*n*\.*\s*|\s+incorporated\.*\s*|\s+company\.*\s*|\s+co\.*\s*","",y)
        y = re.sub(r"\s+services*\s*|\s+associates*\s*|\s+consulting\s*|\s+systems*\s*|\s+technology*i*e*s*\s*","",y)
        y = re.sub(r"\.com\s*|\s+group\.*\s*|\s*n\.*a\.*\s*","",y)
        y = y.strip(",();")
    else:
        y = text
    return y

def fortunefive(text):
    indicator = 0
    text = cleancompanyname(text)
    text = text.split(" ")
    for name in compname:
        name = cleancompanyname(name)
        name = name.split(" ")
        if len(text) == 1 and len(name) == 1:
            text2 = text[0]
            name = name[0]
            ans = 1-nltk.jaccard_distance(set(text2),set(name))
        else:
            ans = 1-nltk.jaccard_distance(set(text),set(name))
        if ans>=0.9:
            
            indicator = 1
            break
    return indicator

cleaned["EMPLOYER_NAME"] = cleaned["EMPLOYER_NAME"].fillna('   ')
cleaned["Fortune"] = cleaned["EMPLOYER_NAME"].apply(fortunefive)
## you can remove Employer name here or manually at the excel
deletecolumns = ['EMPLOYER_NAME']
cleaned.drop(columns = deletecolumns,inplace = True)

In [6]:
test5 = cleaned.copy()
test5.loc[12:,['CASE_STATUS','CASE_SUBMITTED','H-1B_DEPENDENT','AGENT_REPRESENTING_EMPLOYER']]

,CASE_STATUS,CASE_SUBMITTED,H-1B_DEPENDENT,AGENT_REPRESENTING_EMPLOYER
12,DENIED,5,N,N
13,DENIED,3,N,Y
14,DENIED,12,Y,Y
15,DENIED,11,Y,Y
16,DENIED,11,Y,N
...,...,...,...,...
20647,CERTIFIED,7,N,Y
20648,CERTIFIED,8,N,Y
20649,CERTIFIED,3,N,Y
20650,CERTIFIED,3,N,Y


In [7]:
# changing the categorical value into numerical value

def casestatus(x):
    if x == "CERTIFIED":
        return 1
    else:
        return 0

def YorN(x):
    if x == "Y":
        return 1
    elif x == "N":
        return 0

regions = pd.read_csv("USregion.csv",dtype=object)
def findregion(x):
    return float(regions['RegionCode'][regions["STATECODE"] == x])

def annualwage(x):
    x = float(x)
    if x in range (75001):
        return 0
    elif x in range (75000,85001):
        return 1
    else:
        return 2

def totalworker(x):
    x = float(x)
    if x in range(10):
        return 0
    elif x in range(10,50):
        return 1
    elif x in range(50,100):
        return 2
    else:
        return 3
    
def assignquater(x):
    x = float(x)
    if x in [8,9,10]:
        return 0
    elif x in [11,12,1]:
        return 1
    elif x in [2,3,4]:
        return 2
    else:
        return 3

In [8]:
cleaned["CASE_STATUS"] = cleaned["CASE_STATUS"].apply(casestatus)
cleaned["FULL_TIME_POSITION"] = cleaned["FULL_TIME_POSITION"].apply(YorN)
cleaned["TOTAL_WORKER_POSITIONS"] = cleaned["TOTAL_WORKER_POSITIONS"].apply(totalworker)
cleaned["EMPLOYER_REGION"] = cleaned["EMPLOYER_STATE"].apply(findregion)
cleaned.drop(columns = "EMPLOYER_STATE",inplace = True)
cleaned["AGENT_REPRESENTING_EMPLOYER"] = cleaned["AGENT_REPRESENTING_EMPLOYER"].apply(YorN)
cleaned["H-1B_DEPENDENT"] = cleaned["H-1B_DEPENDENT"].apply(YorN)
cleaned["WILLFUL_VIOLATOR"] = cleaned["WILLFUL_VIOLATOR"].apply(YorN)
cleaned["Annual_Wage"] = cleaned["Annual_Wage"].apply(annualwage)
cleaned["CASE_SUBMITTED"] = cleaned["CASE_SUBMITTED"].apply(assignquater)

In [9]:
# Generate Numeric Dataset
cleaned.drop(columns = "CASE_NUMBER",inplace = True)
final = cleaned.copy()
final.to_csv("2019H1BNumericNew1.csv", index=False)
dfff= pd.read_csv('2019H1BNumericNew1.csv')

In [10]:
# Generate One hot encoded data
onehot_data = cleaned.copy()
enc = OneHotEncoder(handle_unknown='ignore') 
enc_df = pd.DataFrame(enc.fit_transform(onehot_data[['EMPLOYER_REGION']]).toarray())
enc_df=enc_df.rename(columns={0: "is_South", 1: "is_West", 2: "is_Northeast", 3: "is_Midwest", 4: "OutOfUS"}, errors="raise")
onehot_data = pd.concat([onehot_data.reset_index(drop=True),enc_df.reset_index(drop=True)], axis=1)
onehot_data.drop(columns = "EMPLOYER_REGION",inplace = True)

enc_df2 = pd.DataFrame(enc.fit_transform(onehot_data[['SOC_CODE']]).toarray())
enc_df2.columns = enc.get_feature_names(['SOC_CODE'])
onehot_data = pd.concat([onehot_data.reset_index(drop=True),enc_df2.reset_index(drop=True)], axis=1)
onehot_data.drop(columns = "SOC_CODE",inplace = True)



In [11]:
final = onehot_data.copy()
final.to_csv("onehot_data.csv", index=False)
MLdata= pd.read_csv('onehot_data.csv')
MLdata.head()

,CASE_STATUS,CASE_SUBMITTED,FULL_TIME_POSITION,TOTAL_WORKER_POSITIONS,AGENT_REPRESENTING_EMPLOYER,H-1B_DEPENDENT,WILLFUL_VIOLATOR,NumOfNaN,PERIOD_OF_EMPLOYMENT_LENGTH,Annual_Wage,...,SOC_CODE_35.0,SOC_CODE_37.0,SOC_CODE_39.0,SOC_CODE_41.0,SOC_CODE_43.0,SOC_CODE_45.0,SOC_CODE_47.0,SOC_CODE_49.0,SOC_CODE_51.0,SOC_CODE_53.0
0,0,3,1,0,1.0,0.0,0.0,5.0,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,1,0,0.0,1.0,0.0,12.0,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,1,0,1.0,0.0,0.0,10.0,3,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0.0,0.0,0.0,18.0,3,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,2,1,0,0.0,0.0,0.0,17.0,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Generate feature selected file from mutual feature (can change to chi2)
for column in MLdata.columns:
    MLdata[column] = MLdata[column].fillna(MLdata[column].mode()[0])
xdata = MLdata.loc[:,MLdata.columns[1:]]
ydata = MLdata.loc[:,['CASE_STATUS']]
xtrain,xtest,ytrain,ytest = train_test_split(xdata,ydata,test_size=0.2,train_size=0.8,shuffle=True,random_state=7406)
# feature selection graph
def select_features(xtrain, ytrain, xtest):
    fs = SelectKBest(score_func=mutual_info_classif, k=10)
    fs.fit(xtrain, ytrain)
    cols = fs.get_support(indices=True)
    X_train_fs = fs.transform(xtrain)
    X_test_fs = fs.transform(xtest)
    return X_train_fs, X_test_fs, fs, cols
X_train_fs, X_test_fs, fs, cols = select_features(xtrain, ytrain, xtest)

bestselect = xtrain.iloc[:,cols]
bestselect2 = xtest.iloc[:,cols]
Final = pd.concat([ytrain.reset_index(drop=True),bestselect.reset_index(drop=True)], axis=1)
Final2 = pd.concat([ytest.reset_index(drop=True),bestselect2.reset_index(drop=True)], axis=1)
Final = Final.append(Final2)
final = Final.copy()
final.to_csv("mutual_10BestSelection.csv", index=False)
df3= pd.read_csv('mutual_10BestSelection.csv')
df3.head()

,CASE_STATUS,CASE_SUBMITTED,H-1B_DEPENDENT,NumOfNaN,Annual_Wage,Fortune,is_West,OutOfUS,SOC_CODE_11.0,SOC_CODE_13.0,...,SOC_CODE_17.0,SOC_CODE_21.0,SOC_CODE_23.0,SOC_CODE_25.0,SOC_CODE_27.0,SOC_CODE_29.0,SOC_CODE_35.0,SOC_CODE_39.0,SOC_CODE_43.0,SOC_CODE_49.0
0,1,1,0.0,4.0,2,0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,0.0,4.0,0,0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2,1.0,13.0,2,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2,0.0,5.0,1,1,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,0.0,15.0,2,1,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
